In [27]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("brickview_database.db")

listings = pd.read_json("listings_20k.json")
property_attributes = pd.read_json("property_attributes_20k.json")
agents = pd.read_json("agents_20k.json")
agents_enhanced = pd.read_json("agents_enhanced_20k.json")
buyers = pd.read_json("buyers_20k.json")
sales = pd.read_csv("sales_20k.csv")

listings = listings.drop_duplicates(subset="Listing_ID")
sales = sales.drop_duplicates(subset="Listing_ID")
property_attributes = property_attributes.drop_duplicates(subset="listing_id")
agents = agents.drop_duplicates(subset="Agent_ID")
agents_enhanced = agents_enhanced.drop_duplicates(subset="agent_id")
buyers = buyers.drop_duplicates(subset="sale_id")

listings.to_sql("listings", conn, if_exists="replace", index=False)
property_attributes.to_sql("property_attributes", conn, if_exists="replace", index=False)
agents.to_sql("agents", conn, if_exists="replace", index=False)
agents_enhanced.to_sql("agents_enhanced", conn, if_exists="replace", index=False)
buyers.to_sql("buyers", conn, if_exists="replace", index=False)
sales.to_sql("sales", conn, if_exists="replace", index=False)

#pd.options.display.float_format = '{:,.1f}'.format

720

In [28]:
# 1. What is the average listing price by city?
query1 = """
SELECT city, ROUND(AVG(price),2) AS Average_Listing_Price
FROM listings
GROUP BY city
ORDER BY Average_Listing_Price DESC
"""

result1 = pd.read_sql(query1, conn)
print("1. Average Listing Price by City:")
print(result1.head())

1. Average Listing Price by City:
          City  Average_Listing_Price
0  Los Angeles             1093038.64
1      Houston             1082539.03
2      Phoenix             1071593.53
3      Chicago             1065799.91
4     New York             1036485.42


In [29]:
# 2. What is the average price per square foot by property type?
query2 = """
SELECT property_type, ROUND(AVG(price/sqft),2) AS Avg_Price_Per_Sqft
FROM listings
GROUP BY property_type
"""

result2 = pd.read_sql(query2, conn)
print("\n2. What is the average price per square foot by property type?")
print(result2.head())


2. What is the average price per square foot by property type?
  Property_Type  Avg_Price_Per_Sqft
0     Apartment              523.30
1         Condo              493.86
2         House              538.93
3     Townhouse              580.42


In [30]:
# 3. How does furnishing status impact property prices?
query3 = """
SELECT furnishing_status AS Furnishing_Status, ROUND(AVG(price),2) AS Prices
FROM listings
JOIN property_attributes
ON listings.Listing_ID = property_attributes.Listing_ID
GROUP BY Furnishing_Status
"""

result3 = pd.read_sql(query3, conn)
print("\n3. How does furnishing status impact property prices?")
print(result3)


3. How does furnishing status impact property prices?
  Furnishing_Status      Prices
0         Furnished  1075380.36
1    Semi-Furnished  1084950.33
2       Unfurnished  1051748.48


In [31]:
# 4. Do properties closer to metro stations command higher prices?
query4 = """
SELECT 
    CASE
        WHEN metro_distance_km < 3 THEN "Close"
        WHEN metro_distance_km >= 3 AND metro_distance_km <= 6 THEN "Moderate"
        WHEN metro_distance_km > 6 THEN "Far Away"
    END AS Category,
    ROUND(AVG(price),2) as Price
FROM property_attributes 
JOIN listings
ON property_attributes.listing_id = listings.listing_id
GROUP BY Category 
"""

result4 = pd.read_sql(query4, conn)
print("\n4. Do properties closer to metro stations command higher prices?")
print(result4)


4. Do properties closer to metro stations command higher prices?
   Category       Price
0     Close  1059397.15
1  Far Away  1078957.57
2  Moderate  1073692.59


In [32]:
# 5. Are rented properties priced differently from non-rented ones?
query5 = """
SELECT 
    CASE
        WHEN is_rented = '1' THEN 'Rented'
        WHEN is_rented = '0' THEN 'Not Rented'
    END AS Rented_Or_Not, 
    ROUND(AVG(price),2) AS Avg_Price
FROM property_attributes 
JOIN listings
ON property_attributes.listing_id = listings.listing_id
GROUP BY Rented_Or_Not
"""

result5 = pd.read_sql(query5, conn)
print("\n5. Are rented properties priced differently from non-rented ones?")
print(result5)


5. Are rented properties priced differently from non-rented ones?
  Rented_Or_Not   Avg_Price
0    Not Rented  1054342.78
1        Rented  1085661.62


In [33]:
# 6. How do bedrooms and bathrooms affect pricing?
query6 = """
SELECT bedrooms, bathrooms, ROUND(AVG(price),2) AS Avg_Price
FROM listings
JOIN property_attributes
ON listings.listing_id = property_attributes.listing_id
GROUP BY bedrooms, bathrooms
"""

result6 = pd.read_sql(query6, conn)
print("\n 6. How do bedrooms and bathrooms affect pricing?")
print(result6)


 6. How do bedrooms and bathrooms affect pricing?
    bedrooms  bathrooms   Avg_Price
0          1          1   999831.38
1          1          2  1036167.89
2          1          3  1021637.07
3          1          4  1086104.70
4          2          1  1143242.80
5          2          2  1070556.14
6          2          3  1134444.43
7          2          4   938737.41
8          3          1  1013173.25
9          3          2  1078875.76
10         3          3  1081313.17
11         3          4  1140171.19
12         4          1  1098157.83
13         4          2  1086079.73
14         4          3  1052435.02
15         4          4  1095472.27
16         5          1   978051.55
17         5          2  1100547.04
18         5          3  1118027.90
19         5          4  1134766.11


In [34]:
# 7. Do properties with parking and power backup sell at higher prices?
query7 = """
SELECT 
    CASE 
        WHEN parking_available = '1' THEN 'Yes'
        WHEN parking_available = '0' THEN 'No'
    END AS Parking_Available,
    CASE 
        WHEN power_backup = '1' THEN 'Yes'
        WHEN power_backup = '0' THEN 'No'
    END AS Power_Backup, 
    ROUND(AVG(price),2) AS Price
FROM property_attributes 
JOIN listings 
ON property_attributes.listing_id = listings.listing_id
GROUP BY Parking_Available, Power_Backup
ORDER BY Parking_Available desc, Power_Backup desc
"""

result7 = pd.read_sql(query7, conn)
print("\n7. Do properties with parking and power backup sell at higher prices?")
print(result7)


7. Do properties with parking and power backup sell at higher prices?
  Parking_Available Power_Backup       Price
0               Yes          Yes  1100052.75
1               Yes           No  1087205.91
2                No          Yes  1044968.22
3                No           No  1048970.77


In [35]:
# 8. How does year built influence listing price?
query8 = """
SELECT Year_Built, ROUND(AVG(price),2) as Avg_Price
FROM property_attributes
JOIN listings 
ON property_attributes.listing_id = listings.listing_id
GROUP BY Year_Built
"""

result8 = pd.read_sql(query8, conn)
print("\n8. How does year built influence listing price?")
print(result8)


8. How does year built influence listing price?
    year_built   Avg_Price
0         1995  1193452.49
1         1996  1026678.20
2         1997  1036903.08
3         1998   983824.37
4         1999  1136310.41
5         2000  1200487.82
6         2001  1089473.45
7         2002  1184247.60
8         2003  1087979.32
9         2004  1133682.39
10        2005   971782.27
11        2006  1130824.67
12        2007  1072989.02
13        2008  1072429.10
14        2009   967706.98
15        2010   948435.59
16        2011  1053879.80
17        2012  1171351.08
18        2013  1006754.83
19        2014   954715.84
20        2015   908974.59
21        2016   972999.58
22        2017  1211992.54
23        2018  1168647.52
24        2019   942260.00
25        2020  1083935.49
26        2021  1206083.68
27        2022  1087743.51
28        2023   896379.59


In [36]:
# 9. Which cities have the highest median property prices?
query9 = """
SELECT city, price
FROM listings
"""

df = pd.read_sql(query9, conn)

result9 = df.groupby('City')['Price'].median().reset_index()
result9.columns = ['City', 'Median_Property_Price']

result9 = result9.sort_values(by='Median_Property_Price', ascending=False)
print("\n9. Which cities have the highest median property prices?")
print(result9)


9. Which cities have the highest median property prices?
          City  Median_Property_Price
2  Los Angeles           1.122531e+06
1      Houston           1.110738e+06
0      Chicago           1.087642e+06
4      Phoenix           1.054334e+06
3     New York           1.029630e+06


In [37]:
# 10. How are properties distributed across price buckets?
query10 = """
SELECT 
    CASE
        WHEN price <= 500000 THEN 'Budget (<= 500000)'
        WHEN price > 500000 and price <= 1000000 THEN 'Mid-Range (> 500000 and <= 1000000)'
        WHEN price > 1000000 and price <= 1500000 THEN 'Premium (> 1000000 and <= 1500000)'
        ELSE 'Luxury (> 1500000)'
    END AS Price_Bucket_Label,
    COUNT(price)
FROM listings 
GROUP BY Price_Bucket_Label
"""

result10 = pd.read_sql(query10, conn)
print("\n10. How are properties distributed across price buckets?")
print(result10)


10. How are properties distributed across price buckets?
                    Price_Bucket_Label  COUNT(price)
0                   Budget (<= 500000)           253
1                   Luxury (> 1500000)           327
2  Mid-Range (> 500000 and <= 1000000)           287
3   Premium (> 1000000 and <= 1500000)           333


In [38]:
# 11. What is the average days on market by city?
query11 = """
SELECT City, ROUND(AVG(Days_On_Market), 2) AS Avg_Days_On_Market
FROM listings 
JOIN sales 
ON listings.Listing_ID = sales.Listing_ID
GROUP BY City
"""

result11 = pd.read_sql(query11, conn)
print("\n11. What is the average days on market by city?")
print(result11)


11. What is the average days on market by city?
          City  Avg_Days_On_Market
0      Chicago               64.25
1      Houston               58.51
2  Los Angeles               65.13
3     New York               60.85
4      Phoenix               59.65


In [39]:
# 12. Which property types sell the fastest?
query12 = """
SELECT Property_Type, ROUND(AVG(Days_On_Market), 2) AS Avg_Days_To_Sell
FROM listings 
JOIN sales 
ON listings.Listing_ID = sales.Listing_ID
GROUP BY Property_Type
ORDER BY Avg_Days_To_Sell ASC
"""

result12 = pd.read_sql(query12, conn)
print("\n12. Which property types sell the fastest?")
print(result12)


12. Which property types sell the fastest?
  Property_Type  Avg_Days_To_Sell
0         House             58.34
1     Apartment             60.65
2     Townhouse             60.96
3         Condo             66.54


In [40]:
# 13. What percentage of properties are sold above listing price?
query13 = """
SELECT 
    ROUND((SUM(CASE 
            WHEN sales.sale_price > listings.price THEN 1 
            ELSE 0 
        END) * 100.0 / COUNT(*)), 2) AS Percentage_Above_Listing
FROM listings 
JOIN sales 
ON listings.listing_id = sales.listing_id
"""

result13 = pd.read_sql(query13, conn)
print("\n13. What percentage of properties are sold above listing price?")
print(result13)


13. What percentage of properties are sold above listing price?
   Percentage_Above_Listing
0                     49.31


In [41]:
# 14. What is the sale-to-list price ratio by city?
query14 = """
SELECT 
    l.city,
    SUM(s.sale_price) / SUM(l.price) AS Sale_To_List_Ratio
FROM listings l
JOIN sales s
ON l.listing_id = s.listing_id
GROUP BY l.city
"""

result14 = pd.read_sql(query14, conn)
print("\n14. What is the sale-to-list price ratio by city?")
print(result14)


14. What is the sale-to-list price ratio by city?
          City  Sale_To_List_Ratio
0      Chicago            1.000395
1      Houston            1.000270
2  Los Angeles            0.998087
3     New York            0.998116
4      Phoenix            0.998462


In [42]:
# 15. Which listings took more than 90 days to sell?
query15 = """
SELECT *
FROM listings 
JOIN sales
ON listings.Listing_ID = sales.Listing_ID
WHERE sales.Days_On_Market > 90
ORDER BY sales.Days_On_Market ASC;
"""

result15 = pd.read_sql(query15, conn)
print("\n15. Which listings took more than 90 days to sell?")
print(result15)


15. Which listings took more than 90 days to sell?
    Listing_ID         City Property_Type         Price         Sqft  \
0       L00992      Houston         House  1.889881e+06  1060.004289   
1       L00568      Chicago         House  1.696925e+06  3716.986525   
2       L00812      Houston         House  3.897840e+05  3880.990478   
3       L00045  Los Angeles         House  4.988960e+05  4892.977499   
4       L00452  Los Angeles         House  9.672980e+05  2961.000986   
..         ...          ...           ...           ...          ...   
185     L00287      Houston     Townhouse  1.724170e+06  3270.995953   
186     L00067      Chicago     Apartment  2.935630e+05  3968.989039   
187     L00259      Phoenix     Apartment  1.513414e+06  1167.993384   
188     L00179      Chicago         Condo  9.713460e+05  4451.992248   
189     L00157      Phoenix     Townhouse  7.108150e+05   767.005110   

    Date_Listed Agent_ID   Latitude   Longitude Listing_ID    Sale_Price  \
0    20

In [43]:
# 16. How does metro distance affect time on market?
query16 = """
SELECT 
    CASE
        WHEN p.metro_distance_km <= 3 THEN 'Near'
        WHEN p.metro_distance_km <= 6 THEN 'Moderate'
        ELSE 'Far Away'
    END AS Distance_Scale,
    ROUND(AVG(s.days_on_market), 2) AS Time_On_Market
FROM property_attributes p
JOIN sales s
ON p.Listing_ID = s.Listing_ID
GROUP BY Distance_Scale
ORDER BY Time_On_Market ASC
"""

result16 = pd.read_sql(query16, conn)
print("\n16. How does metro distance affect time on market?")
print(result16)


16. How does metro distance affect time on market?
  Distance_Scale  Time_On_Market
0       Far Away           59.18
1       Moderate           61.09
2           Near           64.35


In [44]:
# 17. What is the monthly sales trend?
query17 = """
SELECT
    strftime('%Y-%m', Date_Sold) AS Month_Year,
    COUNT(Listing_ID) AS Total_Sales
FROM sales
GROUP BY Month_Year
ORDER BY Month_Year ASC;
"""

result17 = pd.read_sql(query17, conn)
print("\n17. What is the monthly sales trend?")
print(result17)


17. What is the monthly sales trend?
   Month_Year  Total_Sales
0     2023-01            7
1     2023-02           17
2     2023-03           20
3     2023-04           36
4     2023-05           45
5     2023-06           31
6     2023-07           48
7     2023-08           50
8     2023-09           46
9     2023-10           64
10    2023-11           46
11    2023-12           42
12    2024-01           44
13    2024-02           44
14    2024-03           39
15    2024-04           29
16    2024-05           45
17    2024-06           29
18    2024-07           24
19    2024-08           13
20    2024-09            1


In [45]:
# 18. Which properties are currently unsold?
query18 = """
SELECT *
FROM listings l
LEFT JOIN sales s
ON l.Listing_ID = s.Listing_ID
WHERE s.Sale_Price IS NULL
"""

result18 = pd.read_sql(query18, conn)
print("\n18. Which properties are currently unsold?")
print(result18)


18. Which properties are currently unsold?
    Listing_ID         City Property_Type         Price         Sqft  \
0       L00002  Los Angeles     Apartment  1.519141e+06  4966.988193   
1       L00005      Phoenix     Townhouse  5.622970e+05  4178.997421   
2       L00009     New York     Townhouse  1.224551e+06  1522.007780   
3       L00014      Chicago     Apartment  1.377440e+06  1900.998129   
4       L00015  Los Angeles         House  1.069429e+06  3607.995839   
..         ...          ...           ...           ...          ...   
475     L01190      Chicago     Apartment  1.322067e+06   566.003676   
476     L01197  Los Angeles         House  1.243320e+05  1086.989173   
477     L01198  Los Angeles     Townhouse  1.479772e+06  4076.995404   
478     L01199     New York         House  1.446880e+06  3905.995048   
479     L01200      Phoenix     Townhouse  1.407046e+06  1971.983327   

    Date_Listed Agent_ID   Latitude   Longitude Listing_ID Sale_Price  \
0    2023-02-14   

In [46]:
# 19. Which agents have closed the most sales?
query19 = """
SELECT a.Name AS Agent_Name, ae.deals_closed AS Deals_Closed
FROM agents a
JOIN agents_enhanced ae
ON a.Agent_ID = ae.agent_id
ORDER BY ae.deals_closed DESC
"""

result19 = pd.read_sql(query19, conn)
print("\n19. Which agents have closed the most sales?")
print(result19)


19. Which agents have closed the most sales?
     Agent_Name  Deals_Closed
0   Agent A0003           297
1   Agent A0030           296
2   Agent A0010           292
3   Agent A0024           283
4   Agent A0011           270
5   Agent A0048           266
6   Agent A0035           262
7   Agent A0038           257
8   Agent A0044           236
9   Agent A0021           233
10  Agent A0042           231
11  Agent A0016           230
12  Agent A0031           226
13  Agent A0026           225
14  Agent A0009           218
15  Agent A0005           198
16  Agent A0025           188
17  Agent A0020           166
18  Agent A0027           157
19  Agent A0017           156
20  Agent A0040           153
21  Agent A0032           151
22  Agent A0039           142
23  Agent A0046           135
24  Agent A0007           127
25  Agent A0050           109
26  Agent A0045            99
27  Agent A0036            98
28  Agent A0047            91
29  Agent A0018            89
30  Agent A0043         

In [47]:
# 20. Who are the top agents by total sales revenue?
query20 = """
SELECT a.Name AS Agent_Name, ROUND(SUM(s.Sale_Price), 2) AS Total_Sales_Revenue
FROM listings l
JOIN sales s ON l.Listing_ID = s.Listing_ID
JOIN agents a ON a.Agent_ID = l.Agent_ID
GROUP BY A.Name
ORDER BY Total_Sales_Revenue DESC
"""

result20 = pd.read_sql(query20, conn)
print("\n20. Who are the top agents by total sales revenue?")
print(result20)


20. Who are the top agents by total sales revenue?
     Agent_Name  Total_Sales_Revenue
0   Agent A0011          27882272.02
1   Agent A0042          27191605.99
2   Agent A0043          24102418.02
3   Agent A0035          22725751.99
4   Agent A0014          22034008.02
5   Agent A0046          21335804.97
6   Agent A0048          21186306.00
7   Agent A0027          21099696.96
8   Agent A0009          20279274.97
9   Agent A0029          19586498.02
10  Agent A0039          19223013.98
11  Agent A0036          19209543.94
12  Agent A0050          18968547.96
13  Agent A0025          18818591.05
14  Agent A0017          18583851.04
15  Agent A0007          18414480.00
16  Agent A0001          18173919.98
17  Agent A0015          17752856.04
18  Agent A0019          16730611.98
19  Agent A0021          16548308.01
20  Agent A0006          16523231.01
21  Agent A0031          15723293.00
22  Agent A0003          15584954.00
23  Agent A0018          15539876.00
24  Agent A0012        

In [48]:
# 21. Which agents close deals fastest?
query21 = """
SELECT a.agent_id AS Agent_ID , ae.avg_closing_days AS Avg_Closing_Days
FROM agents_enhanced ae
JOIN agents a ON a.Agent_ID = ae.agent_id
ORDER BY avg_closing_days ASC
"""

result21 = pd.read_sql(query21, conn)
print("\n21. Which agents close deals fastest?")
print(result21)


21. Which agents close deals fastest?
   Agent_ID  Avg_Closing_Days
0     A0048                15
1     A0035                20
2     A0042                21
3     A0023                23
4     A0045                26
5     A0041                27
6     A0016                31
7     A0034                31
8     A0028                33
9     A0032                33
10    A0047                33
11    A0049                33
12    A0044                35
13    A0027                36
14    A0009                38
15    A0018                39
16    A0026                42
17    A0022                44
18    A0025                48
19    A0006                49
20    A0040                52
21    A0029                56
22    A0033                57
23    A0017                58
24    A0046                58
25    A0011                60
26    A0031                60
27    A0004                61
28    A0019                62
29    A0001                64
30    A0013                65
3

In [49]:
# 22. Does experience correlate with deals closed?
query22 = """
SELECT 
    CASE
        WHEN experience_years <= 5 THEN '0-5 Years'
        WHEN experience_years <= 10 THEN '6-10 Years'
        WHEN experience_years <= 15 THEN '11-15 Years'
        WHEN experience_years <= 20 THEN '16-20 Years'
        ELSE 'More than 20 Years'
    END AS Experience_Category,
    SUM(deals_closed) AS Total_Deals_Closed
FROM agents_enhanced 
GROUP BY Experience_Category 
ORDER BY Total_Deals_Closed DESC
"""

result22 = pd.read_sql(query22, conn)
print("\n22. Does experience correlate with deals closed?")
print(result22)


22. Does experience correlate with deals closed?
  Experience_Category  Total_Deals_Closed
0         16-20 Years                2270
1  More than 20 Years                1664
2         11-15 Years                1411
3           0-5 Years                 871
4          6-10 Years                 616


In [50]:
# 23. Do agents with higher ratings close deals faster?
query23 = """
SELECT
    CASE
        WHEN Rating >= 4.5 THEN '1. 4.5 - 5.0 (Excellent)'
        WHEN Rating >= 4.0 THEN '2. 4.0 - 4.4 (Very Good)'
        WHEN Rating >= 3.5 THEN '3. 3.5 - 3.9 (Good)'
        WHEN Rating >= 3.0 THEN '4. 3.0 - 3.4 (Average)'
        ELSE 'Below 3.0 (5. Low)'
    END AS Rating_Category,
    ROUND(AVG(Avg_Closing_Days), 3) AS Avg_Closing_Time
FROM agents_enhanced
GROUP BY Rating_Category
ORDER BY Avg_Closing_Time ASC;
"""

result23 = pd.read_sql(query23, conn)
print("\n23. Do agents with higher ratings close deals faster?")
print(result23)


23. Do agents with higher ratings close deals faster?
            Rating_Category  Avg_Closing_Time
0  2. 4.0 - 4.4 (Very Good)            48.375
1  1. 4.5 - 5.0 (Excellent)            51.700
2    4. 3.0 - 3.4 (Average)            56.750
3       3. 3.5 - 3.9 (Good)            67.667


In [51]:
# 24. What is the average commission earned by each agent?
query24 = """
SELECT a.Name, ROUND(AVG(s.Sale_Price * ae.commission_rate * 0.01), 2) AS Average_Commission_Earned
FROM agents_enhanced ae
JOIN agents a ON ae.agent_id = a.agent_id
JOIN listings l ON a.agent_id = l.agent_id
JOIN sales s ON s.Listing_ID = l.listing_id
GROUP BY a.Name
"""

result24 = pd.read_sql(query24, conn)
print("\n24. What is the average commission earned by each agent?")
print(result24)


24. What is the average commission earned by each agent?
           Name  Average_Commission_Earned
0   Agent A0001                   24231.89
1   Agent A0002                   32326.16
2   Agent A0003                   27760.70
3   Agent A0004                   19364.05
4   Agent A0005                   11226.97
5   Agent A0006                   17404.47
6   Agent A0007                   17251.46
7   Agent A0008                   17312.42
8   Agent A0009                   33997.61
9   Agent A0010                   20298.85
10  Agent A0011                   23351.40
11  Agent A0012                   31737.61
12  Agent A0013                   23176.67
13  Agent A0014                   20670.00
14  Agent A0015                   16532.35
15  Agent A0016                   13002.01
16  Agent A0017                   28406.74
17  Agent A0018                   31438.36
18  Agent A0019                   27556.30
19  Agent A0020                   25646.69
20  Agent A0021                   26574

In [52]:
# 25. Which agents currently have the most active listings?
query25 = """
SELECT a.Name AS Agents_Name, count(*) AS Active_Listings
FROM listings l
JOIN agents_enhanced ae ON l.Agent_ID = ae.agent_id
JOIN agents a ON l.Agent_ID = a.agent_id
LEFT JOIN sales s ON l.Listing_ID = s.Listing_ID
WHERE s.Sale_Price IS NULL
GROUP BY Agents_Name
ORDER BY Active_Listings DESC
"""

result25 = pd.read_sql(query25, conn)
print("\n25. Which agents currently have the most active listings?")
print(result25)


25. Which agents currently have the most active listings?
    Agents_Name  Active_Listings
0   Agent A0043               18
1   Agent A0026               16
2   Agent A0035               15
3   Agent A0018               15
4   Agent A0044               13
5   Agent A0025               13
6   Agent A0016               13
7   Agent A0009               13
8   Agent A0004               13
9   Agent A0003               13
10  Agent A0041               12
11  Agent A0022               12
12  Agent A0007               12
13  Agent A0049               11
14  Agent A0045               11
15  Agent A0039               11
16  Agent A0038               11
17  Agent A0034               11
18  Agent A0048               10
19  Agent A0042               10
20  Agent A0037               10
21  Agent A0036               10
22  Agent A0029               10
23  Agent A0023               10
24  Agent A0006               10
25  Agent A0040                9
26  Agent A0020                9
27  Agent A0014  

In [53]:
# 26. What percentage of buyers are investors vs end users?
query26 = """
SELECT 
    buyer_type AS Buyer_Type, (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM buyers)) AS Percentage_Of_Buyers
FROM buyers 
GROUP BY buyer_type
ORDER BY buyer_type DESC
"""

result26 = pd.read_sql(query26, conn)
print("\n26. What percentage of buyers are investors vs end users?")
print(result26)


26. What percentage of buyers are investors vs end users?
  Buyer_Type  Percentage_Of_Buyers
0   Investor             48.333333
1   End User             51.666667


In [54]:
# 27. Which cities have the highest loan uptake rate?
query27 = """
SELECT l.City, ROUND((COUNT(*) * 100.0 / 356), 2) AS Loan_Uptake_Rate
FROM buyers b
JOIN listings l ON b.sale_id = l.Listing_ID
WHERE b.loan_taken = 1
GROUP BY l.City
ORDER BY Loan_Uptake_Rate DESC
"""

result27 = pd.read_sql(query27, conn)
print("\n27. Which cities have the highest loan uptake rate?")
print(result27)


27. Which cities have the highest loan uptake rate?
          City  Loan_Uptake_Rate
0      Chicago             22.19
1      Phoenix             21.91
2      Houston             20.51
3     New York             19.10
4  Los Angeles             16.29


In [55]:
# 28. What is the average loan amount by buyer type?
query28 = """
SELECT buyer_type AS Buyer_Type, ROUND(AVG(loan_amount),2) AS Average_Loan_Amount
FROM buyers 
GROUP BY Buyer_Type
"""

result28 = pd.read_sql(query28, conn)
print("\n28. What is the average loan amount by buyer type? ")
print(result28)


28. What is the average loan amount by buyer type? 
  Buyer_Type  Average_Loan_Amount
0   End User           2622613.68
1   Investor           2456486.32


In [56]:
# 29. Which payment mode is most commonly used?
query29 = """
SELECT payment_mode AS Payment_Mode, COUNT(*) AS Usage_Count
FROM buyers
GROUP BY Payment_Mode
ORDER BY Usage_Count DESC
"""

result29 = pd.read_sql(query29, conn)
print("\n29. Which payment mode is most commonly used?")
print(result29)


29. Which payment mode is most commonly used?
    Payment_Mode  Usage_Count
0            UPI          188
1           Cash          183
2         Cheque          181
3  Bank Transfer          168


In [57]:
# 30. Do loan-backed purchases take longer to close?
query30 = """
SELECT 
    CASE
        WHEN b.loan_taken = 0 THEN 'Loan Not Taken'
        ELSE 'Loan Taken'
    END AS Loan_Taken_Or_Not,
    ROUND(AVG(s.Days_On_Market), 2) AS Avg_Days_To_Close
FROM sales s
JOIN buyers b ON s.Listing_ID = b.sale_id
GROUP BY b.loan_taken;
"""

result30 = pd.read_sql(query30, conn)
print("\n30. Do loan-backed purchases take longer to close?")
print(result30)


30. Do loan-backed purchases take longer to close?
  Loan_Taken_Or_Not  Avg_Days_To_Close
0    Loan Not Taken              61.23
1        Loan Taken              62.06


In [58]:
conn.close()